In [1]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelBinarizer
import os

# go to project root dir
os.chdir("../..")

Use spotify feature vector for each track to predict the term:

In [60]:
split = pd.read_csv("tmp/track_train_test_split.csv")
data = pd.read_csv("tmp/track_features.csv")
features = ["danceability", "energy", "valence", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness"]

train_X = data.merge(split).query("is_train").set_index("id")[features]
train_Y = data.merge(split).query("is_train").term
train_Y = LabelBinarizer().fit_transform(train_Y)

test_X = data.merge(split).query("not is_train").set_index("id")[features]
test_Y = data.merge(split).query("not is_train").term
test_Y = LabelBinarizer().fit_transform(test_Y)


print(f"{len(features)} features and {len(train_Y)} samples and {train_Y.shape[1]} classes")


8 features and 479 samples and 3 classes


In [61]:
from keras.layers import Input, Dense, Dropout

model = keras.Sequential(
    [
        Input(shape=(len(features),)),
        Dense(10, activation="relu"),
        #Dropout(0.2),
        Dense(10, activation="relu"), 
        Dense(train_Y.shape[1], activation="softmax"),
    ]
)
model.build()
model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=[keras.metrics.CategoricalAccuracy(), keras.metrics.TopKCategoricalAccuracy(k=2)]
)
model.summary()

tensorboard_callback = keras.callbacks.TensorBoard(log_dir="tmp/tensorboartd", histogram_freq=2)
model.fit(train_X, train_Y, validation_data=(test_X, test_Y), callbacks=[tensorboard_callback])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 10)                90        
                                                                 
 dense_29 (Dense)            (None, 10)                110       
                                                                 
 dense_30 (Dense)            (None, 3)                 33        
                                                                 
Total params: 233
Trainable params: 233
Non-trainable params: 0
_________________________________________________________________
15/15 [==============================] - 1s 15ms/step - loss: 1.1287 - categorical_accuracy: 0.3925 - top_k_categorical_accuracy: 0.6660 - val_loss: 1.0144 - val_categorical_accuracy: 0.3750 - val_top_k_categorical_accuracy: 0.6667


In [62]:
model.evaluate(test_X, test_Y)
model.predict(test_X)

4/4 [==============================] - 0s 2ms/step


array([[0.46597782, 0.18756427, 0.34645778],
       [0.51823145, 0.15607356, 0.32569498],
       [0.42634127, 0.19830456, 0.37535414],
       [0.4524756 , 0.20097543, 0.346549  ],
       [0.40130898, 0.23792425, 0.36076686],
       [0.4451328 , 0.17811406, 0.37675303],
       [0.45430964, 0.17884599, 0.3668444 ],
       [0.2738922 , 0.29494095, 0.43116686],
       [0.48818183, 0.17372109, 0.33809698],
       [0.49148235, 0.15017772, 0.3583399 ],
       [0.4962831 , 0.1738139 , 0.329903  ],
       [0.4718964 , 0.15833642, 0.3697672 ],
       [0.4064656 , 0.22312197, 0.37041235],
       [0.5908799 , 0.09463205, 0.31448805],
       [0.42640537, 0.20558625, 0.36800835],
       [0.4154112 , 0.21021062, 0.37437817],
       [0.49851614, 0.1732856 , 0.32819825],
       [0.5333786 , 0.1380718 , 0.32854953],
       [0.582347  , 0.09210988, 0.32554317],
       [0.5094297 , 0.14265296, 0.34791735],
       [0.33125338, 0.29960334, 0.36914334],
       [0.39641818, 0.24783786, 0.355744  ],
       [0.